# Naive Bayes

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler  # PaRA balancear los datos
import re
import nltk
from nltk.corpus import stopwords

#stopwords en español
nltk.download('stopwords')

# Cargar dataset desde la URL
url = "https://raw.githubusercontent.com/ignaciomsarmiento/RecomSystemsLectures/main/L07_sentimientos/data/Amazon.csv"
df = pd.read_csv(url)

# Comentarios en español
df = df[['reviews.text_esp', 'reviews.rating']]

# Eliminar filas con valores nulos
df = df.dropna(subset=['reviews.text_esp', 'reviews.rating'])

# Asignar etiquetas de sentimiento según la puntuacion (solo positivo y negativo)
df['sentimiento'] = df['reviews.rating'].apply(lambda x: 'negativo' if x <= 2 else 'positivo')

# Filtrar los comentarios con mas de 3 palabras
df = df[df['reviews.text_esp'].apply(lambda x: isinstance(x, str) and len(x.split()) > 3)]

# Preprocesamiento de texto con eliminacion de stopwords
def preprocesar_texto(texto):
    # Convertir todo a minusculas
    texto = texto.lower()
    
    # Eliminar caracteres no alfabeticos
    texto = re.sub(r'[^a-záéíóúñ\s]', '', texto)
    
    # Eliminar stopwords en español
    stop_words = set(stopwords.words('spanish'))
    texto = ' '.join([palabra for palabra in texto.split() if palabra not in stop_words])
    
    return texto

# Aplicar preprocesamiento a los comentarios
df['reviews.text_esp'] = df['reviews.text_esp'].apply(preprocesar_texto)

# Dividir el dataset en conjunto de entrenamiento y prueba
X = df['reviews.text_esp']
y = df['sentimiento']

# Balancear las clases en el conjunto de entrenamiento utilizando sobremuestreo
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X.values.reshape(-1, 1), y)

# Convertir X_res de nuevo a formato adecuado
X_res = X_res.flatten()

# Dividir los datos balanceados en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Crear un pipeline con CountVectorizer y un clasificador Naive Bayes
pipeline = make_pipeline(CountVectorizer(), MultinomialNB())

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy de Naive Bayes: {accuracy:.4f}')

# Mostrar algunos resultados de prediccion
print("\nAlgunas predicciones de Naive Bayes:")
for text, true_label, pred_label in zip(X_test[:10], y_test[:10], y_pred[:10]):
    print(f"Comentario: {text[:100]}... | Sentimiento Real: {true_label} | Predicción: {pred_label}")

# Reporte de clasificacion de Naive Bayes
print("\nReporte de clasificación de Naive Bayes:")
print(classification_report(y_test, y_pred))


[nltk_data] Downloading package stopwords to C:\Users\Jesus De La
[nltk_data]     Fuente\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy de Naive Bayes: 0.8167

Algunas predicciones de Naive Bayes:
Comentario: así satisfecho elección amazon incluir solo cargador esencialmente inútil comprar kindle hd cedí ped... | Sentimiento Real: negativo | Predicción: negativo
Comentario: kindle sigue congelándose ver película congelará aleatoriamente... | Sentimiento Real: negativo | Predicción: positivo
Comentario: producto pagas conseguí viernes negro menos dólares usos básicos buenos hijos si necesita niñera ent... | Sentimiento Real: positivo | Predicción: positivo
Comentario: bien siempre espere cosas aplicaciones juegos samsung apple así propia linea... | Sentimiento Real: negativo | Predicción: negativo
Comentario: siempre ido comprar kindle gustan libros juegos disponibles encontré impredecibles confiables color ... | Sentimiento Real: negativo | Predicción: negativo
Comentario: encanta altavoz grande funciona genial utilizar fuente música principalmente... | Sentimiento Real: positivo | Predicción: positivo
Comenta